In [13]:
from nba_api.live.nba.endpoints import scoreboard

# Get the scoreboard for the specified date
scores = scoreboard.ScoreBoard()

# Extract the games data
games = scores.get_dict()['scoreboard']['games']

# Print information about each game
for game in games:
    home_team = game['homeTeam']['teamName']
    away_team = game['awayTeam']['teamName']
    game_status = game['gameStatus']
    
    print(f"{away_team} @ {home_team}")
    print(f"Status: {game_status}")
    print(game)
    
    if game_status == 1:  # Game hasn't started yet
        home_score = game['homeTeam']['score']
        away_score = game['awayTeam']['score']
        
    
    print("\n")
print(games)

Nuggets @ Celtics
Status: 3
{'gameId': '0012400005', 'gameCode': '20241006/DENBOS', 'gameStatus': 3, 'gameStatusText': 'Final', 'period': 4, 'gameClock': '', 'gameTimeUTC': '2024-10-06T14:00:00Z', 'gameEt': '2024-10-06T10:00:00Z', 'regulationPeriods': 4, 'ifNecessary': False, 'seriesGameNumber': '', 'gameLabel': 'Preseason', 'gameSubLabel': 'NBA Abu Dhabi Game', 'seriesText': 'Neutral Site', 'seriesConference': '', 'poRoundDesc': '', 'gameSubtype': 'Global Games', 'homeTeam': {'teamId': 1610612738, 'teamName': 'Celtics', 'teamCity': 'Boston', 'teamTricode': 'BOS', 'wins': 2, 'losses': 0, 'score': 130, 'seed': None, 'inBonus': None, 'timeoutsRemaining': 0, 'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 34}, {'period': 2, 'periodType': 'REGULAR', 'score': 33}, {'period': 3, 'periodType': 'REGULAR', 'score': 42}, {'period': 4, 'periodType': 'REGULAR', 'score': 21}]}, 'awayTeam': {'teamId': 1610612743, 'teamName': 'Nuggets', 'teamCity': 'Denver', 'teamTricode': 'DEN', 'wins': 

In [15]:
from nba_api.stats.endpoints import teamgamelog
import pandas as pd

def get_team_game_log(team_id, season):
    # Get the team game log
    game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
    
    # Convert to DataFrame
    df = game_log.get_data_frames()[0]
    return df

# Get the team ID (e.g., Golden State Warriors)
team_id = '1610612744'  # You can get this from nba_api.stats.static.teams

# Specify the season (e.g., 2022-23 season)
season = '2022-23'

# Get the team game log
game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)

# Convert to DataFrame
df = game_log.get_data_frames()[0]
# Display the first few rows
print(df.head())

# Get basic statistics
print(df['WL'].value_counts())  # Count of wins and losses
print(df['PTS'].mean())  # Average points per game

      Team_ID     Game_ID     GAME_DATE      MATCHUP WL   W   L  W_PCT  MIN  \
0  1610612744  0022201230  APR 09, 2023    GSW @ POR  W  44  38  0.537  240   
1  1610612744  0022201211  APR 07, 2023    GSW @ SAC  W  43  38  0.531  240   
2  1610612744  0022201187  APR 04, 2023  GSW vs. OKC  W  42  38  0.525  240   
3  1610612744  0022201175  APR 02, 2023    GSW @ DEN  L  41  38  0.519  240   
4  1610612744  0022201158  MAR 31, 2023  GSW vs. SAS  W  41  37  0.526  240   

   FGM  ...  FT_PCT  OREB  DREB  REB  AST  STL  BLK  TOV  PF  PTS  
0   58  ...   0.875     9    49   58   47   13    6   15  18  157  
1   41  ...   0.875    10    37   47   36    6    7   24  22  119  
2   47  ...   0.733    19    35   54   25    6    6   13  22  136  
3   42  ...   1.000    22    35   57   26    9    4   15  22  110  
4   47  ...   0.714    14    31   45   37    5    4   14  18  130  

[5 rows x 27 columns]
WL
W    44
L    38
Name: count, dtype: int64
118.9390243902439


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Function to get the matchups 
def get_team_game_log(team_id, season):
    game_log = teamgamelog.TeamGameLog(team_id=team_id, season=season)
    df = game_log.get_data_frames()[0]
    return df

# Get the team ID and season
team_id = '1610612744'  # Golden State Warriors
season = '2022-23'

# Fetch the data 
df = get_team_game_log(team_id, season)

# Select features and target variable
features = df[['FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 
               'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']]
target = df['WL']

# Encode target variable (Win = 1, Loss = 0)
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the MLP model
mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, activation='relu', solver='adam', random_state=42)

# Train the model
mlp_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = mlp_model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(report)


Accuracy: 0.71
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.44      0.62         9
           1       0.62      1.00      0.76         8

    accuracy                           0.71        17
   macro avg       0.81      0.72      0.69        17
weighted avg       0.82      0.71      0.68        17



c:\Users\nicho\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
